In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from skimage.util import img_as_ubyte
import torchvision.transforms.functional as TF

import timm
import torch.nn as nn
from torchinfo import summary
from tqdm import tqdm

import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

import gc

In [2]:
batch_size = 32
os.makedirs('outputs', exist_ok=True)

# Dataset

## Load

In [3]:
def loading_the_data(data_dir):
    # Generate data paths with labels
    filepaths = []
    labels = []

    # Get folder names
    folds = os.listdir(data_dir)

    for fold in folds:
        foldpath = os.path.join(data_dir, fold)
        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath, file)
            
            filepaths.append(fpath)
            labels.append(fold)

    # Concatenate data paths with labels into one DataFrame
    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')

    df = pd.concat([Fseries, Lseries], axis=1)
    
    return df

In [4]:
dir = 'G:\\FireRisk'

train_df = loading_the_data(dir + '\\train')
test_df = loading_the_data(dir + '\\val')

train_df

,filepaths,labels
0,G:\FireRisk\train\High\27032281_4_-103.4304412...,High
1,G:\FireRisk\train\High\27038991_4_-77.77273442...,High
2,G:\FireRisk\train\High\27040201_4_-73.83896834...,High
3,G:\FireRisk\train\High\27042071_4_-122.1662712...,High
4,G:\FireRisk\train\High\27042401_4_-121.1231610...,High
...,...,...
70326,G:\FireRisk\train\Water\35471591_7_-72.4088150...,Water
70327,G:\FireRisk\train\Water\35484351_7_-82.8478475...,Water
70328,G:\FireRisk\train\Water\35487101_7_-72.1588909...,Water
70329,G:\FireRisk\train\Water\35497331_7_-90.9452064...,Water


## Menambahkan Label Kontinu

In [5]:
cnt_df = pd.read_csv("conversion_cnt.csv")
train_df = pd.merge(train_df, cnt_df[['filepaths', 'grid_code']], on='filepaths', how='left')
train_df.rename(columns={'grid_code': 'labels_cnt'}, inplace=True)

cnt_df = pd.read_csv("conversion_test_cnt.csv")
test_df = pd.merge(test_df, cnt_df[['filepaths', 'grid_code']], on='filepaths', how='left')
test_df.rename(columns={'grid_code': 'labels_cnt'}, inplace=True)

train_df

,filepaths,labels,labels_cnt
0,G:\FireRisk\train\High\27032281_4_-103.4304412...,High,1237
1,G:\FireRisk\train\High\27038991_4_-77.77273442...,High,628
2,G:\FireRisk\train\High\27040201_4_-73.83896834...,High,718
3,G:\FireRisk\train\High\27042071_4_-122.1662712...,High,805
4,G:\FireRisk\train\High\27042401_4_-121.1231610...,High,1093
...,...,...,...
70326,G:\FireRisk\train\Water\35471591_7_-72.4088150...,Water,0
70327,G:\FireRisk\train\Water\35484351_7_-82.8478475...,Water,0
70328,G:\FireRisk\train\Water\35487101_7_-72.1588909...,Water,0
70329,G:\FireRisk\train\Water\35497331_7_-90.9452064...,Water,0


In [6]:
# cls_df = pd.read_csv("conversion_cls_0025.csv")
# mismatch_df = cls_df[cls_df['labels'] != cls_df['label']]

# print(f"Number of mismatches: {len(mismatch_df)}")
# mismatch_df

In [7]:
# train_df.iloc[10:15]

In [8]:
# train_df.loc[train_df['filepaths'].isin(mismatch_df['filepaths']), 'labels_cnt'] = 0
# train_df.iloc[10:15]

## Encoding Label Kelas

In [9]:
class_names = ['Water', 'Non-burnable', 'Very_Low', 'Low', 'Moderate', 'High', 'Very_High']
label_encoder = OrdinalEncoder(categories=[class_names])

train_df['labels'] = label_encoder.fit_transform(train_df[['labels']])
test_df['labels'] = label_encoder.transform(test_df[['labels']])

train_df['labels'] = train_df['labels'].astype('int64')
test_df['labels'] = test_df['labels'].astype('int64')

train_df

,filepaths,labels,labels_cnt
0,G:\FireRisk\train\High\27032281_4_-103.4304412...,5,1237
1,G:\FireRisk\train\High\27038991_4_-77.77273442...,5,628
2,G:\FireRisk\train\High\27040201_4_-73.83896834...,5,718
3,G:\FireRisk\train\High\27042071_4_-122.1662712...,5,805
4,G:\FireRisk\train\High\27042401_4_-121.1231610...,5,1093
...,...,...,...
70326,G:\FireRisk\train\Water\35471591_7_-72.4088150...,0,0
70327,G:\FireRisk\train\Water\35484351_7_-82.8478475...,0,0
70328,G:\FireRisk\train\Water\35487101_7_-72.1588909...,0,0
70329,G:\FireRisk\train\Water\35497331_7_-90.9452064...,0,0


## Normalisasi Label Kontinu

In [10]:
def normalize_cont_label(label):
    if label <= 0:
        return 0
    elif label <= 61:
        return label / 61
    elif 61 < label <= 178:
        return (label - 61) / (178 - 61) + 1
    elif 178 < label <= 489:
        return (label - 178) / (489 - 178) + 2
    elif 489 < label <= 1985:
        return (label - 489) / (1985 - 489) + 3
    elif 1985 < label:
        return (label - 1985) / (100000 - 1985) + 4

In [11]:
train_df['labels_cnt'] = train_df['labels_cnt'].apply(normalize_cont_label).round(3)
test_df['labels_cnt'] = test_df['labels_cnt'].apply(normalize_cont_label).round(3)

train_df

,filepaths,labels,labels_cnt
0,G:\FireRisk\train\High\27032281_4_-103.4304412...,5,3.500
1,G:\FireRisk\train\High\27038991_4_-77.77273442...,5,3.093
2,G:\FireRisk\train\High\27040201_4_-73.83896834...,5,3.153
3,G:\FireRisk\train\High\27042071_4_-122.1662712...,5,3.211
4,G:\FireRisk\train\High\27042401_4_-121.1231610...,5,3.404
...,...,...,...
70326,G:\FireRisk\train\Water\35471591_7_-72.4088150...,0,0.000
70327,G:\FireRisk\train\Water\35484351_7_-82.8478475...,0,0.000
70328,G:\FireRisk\train\Water\35487101_7_-72.1588909...,0,0.000
70329,G:\FireRisk\train\Water\35497331_7_-90.9452064...,0,0.000


In [12]:
train_df.describe()

,labels,labels_cnt
count,70331.000000,70331.000000
mean,2.547156,1.109816
std,1.498586,1.271991
min,0.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.557000
75%,4.000000,2.035000
max,6.000000,4.576000


## Split Data

In [13]:
# unused = 0.8
# train_df, unused_df = train_test_split(train_df, test_size = unused, shuffle = True, random_state = 49, stratify=train_df['labels'])
# test_df, unused_df = train_test_split(test_df, test_size = unused, shuffle = True, random_state = 49, stratify=test_df['labels'])

In [14]:
train_df, valid_df = train_test_split(train_df, test_size = 0.2, shuffle = True, random_state = 49, stratify=train_df['labels'])

## Augmentasi

In [15]:
class FireRiskDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        # Return the number of samples in the dataset
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get the file path and label for the index
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        label_cnt = self.dataframe.iloc[idx, 2]
        
        # Open the image
        image = Image.open(img_path).convert("RGB")

        # If there is any transform (e.g., normalization, augmentation), apply it
        if self.transform:
            image = self.transform(image)

        return image, label, label_cnt, img_path

In [16]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    # transforms.CenterCrop(224),  # Crop image to get 224x224 in the center
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

augment = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.RandomHorizontalFlip(p=0.2),  # Randomly flip image horizontally
    transforms.RandomAffine(degrees=10, translate=(0.03125, 0.03125), fill=(0, 0, 0)),  # Random affine transformations (rotation, translation)
    # transforms.CenterCrop(224),  # Crop image to get 224x224 in the center
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

# Create the dataset
train_dataset = FireRiskDataset(dataframe=train_df, transform=augment)
valid_dataset = FireRiskDataset(dataframe=valid_df, transform=transform)
test_dataset = FireRiskDataset(dataframe=test_df, transform=transform)

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
# def imshow(img):
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))  # Convert CHW to HWC format
#     plt.show()

# # Get a batch of training data and displaying it
# data_iter = iter(train_loader)
# images, labels, labels_cnt, _ = next(data_iter)
# imshow(torchvision.utils.make_grid(images[:4]))

# Model

In [18]:
if 'mae_model' in globals() and mae_model != None:
    mae_model.cpu()
    del mae_model
torch.cuda.empty_cache()
gc.collect()

36

In [19]:
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## MAE Model - Latent Extraction

In [20]:
# Load MAE model
mae_model = timm.create_model('vit_base_patch16_224', pretrained=False)

# Load the pre-trained weights
checkpoint = torch.load('mae_pretrain_vit_base.pth', weights_only=True)
state_dict = checkpoint['model'] if 'model' in checkpoint else checkpoint
mae_model.load_state_dict(state_dict, strict=False)

# Remove the final classification head (to use only the encoder part)
mae_model.reset_classifier(0)
mae_model = mae_model.to(device)

In [21]:
summary(mae_model, input_size=(batch_size, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
VisionTransformer                        [32, 768]                 152,064
├─PatchEmbed: 1-1                        [32, 196, 768]            --
│    └─Conv2d: 2-1                       [32, 768, 14, 14]         590,592
│    └─Identity: 2-2                     [32, 196, 768]            --
├─Dropout: 1-2                           [32, 197, 768]            --
├─Identity: 1-3                          [32, 197, 768]            --
├─Identity: 1-4                          [32, 197, 768]            --
├─Sequential: 1-5                        [32, 197, 768]            --
│    └─Block: 2-3                        [32, 197, 768]            --
│    │    └─LayerNorm: 3-1               [32, 197, 768]            1,536
│    │    └─Attention: 3-2               [32, 197, 768]            2,362,368
│    │    └─Identity: 3-3                [32, 197, 768]            --
│    │    └─Identity: 3-4                [32, 197, 768]          

## Latent Extraction

In [22]:
def extract_texture_features(image_tensor):
    image_cpu = image_tensor.to('cpu')
    img_gray = TF.to_pil_image(image_cpu).convert('L')
    img_gray = img_as_ubyte(np.array(img_gray))

    # GLCM
    glcm = graycomatrix(img_gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    ASM = graycoprops(glcm, 'ASM')[0, 0]
    glcm_feat = [contrast, dissimilarity, homogeneity, energy, correlation, ASM]

    # LBP
    lbp = local_binary_pattern(img_gray, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)

    return glcm_feat, hist.tolist()

In [23]:
def extract_latent_representations(dataloader, model, device, epoch=1):
    model.eval()
    latent_representations = []
    multi_latents = []
    labels = []
    labels_cnt = []
    filenames = []

    glcm_features = []
    multi_glcm = []
    lbp_features = []
    multi_lbp = []

    with torch.no_grad():
        for images, targets, targets_cnt, filename in tqdm(dataloader, unit="batch"):
            images = images.to(device)

            # Forward pass through the MAE encoder
            latent = model(images)
            latent_representations.append(latent.cpu())

            # Extract GLCM and LBP
            for i in range(images.size(0)):
                glcm_feat, lbp_feat = extract_texture_features(images[i])
                glcm_features.append(glcm_feat)
                lbp_features.append(lbp_feat)

            labels.extend(targets)
            labels_cnt.extend(targets_cnt)
            filenames.extend(filename)

    # Concatenate the results across batches
    latent_representations = torch.cat(latent_representations, dim=0)
    glcm_features = torch.tensor(glcm_features)
    lbp_features = torch.tensor(lbp_features)
    
    if epoch == 1:
        return latent_representations, glcm_features, lbp_features, labels, labels_cnt, filenames
    
    multi_latents.append(latent_representations)
    multi_glcm.append(glcm_features)
    multi_lbp.append(lbp_features)
    
    while epoch > 1:
        latent_representations = []
        glcm_features = []
        lbp_features = []
        with torch.no_grad():
            for images, targets, targets_cnt, filename in tqdm(dataloader, unit="batch"):
                images = images.to(device)
    
                # Forward pass through the MAE encoder
                latent = model(images)
                latent_representations.append(latent.cpu())

                # Extract GLCM and LBP
                for i in range(images.size(0)):
                    glcm_feat, lbp_feat = extract_texture_features(images[i])
                    glcm_features.append(glcm_feat)
                    lbp_features.append(lbp_feat)
    
        # Concatenate the results across batches
        latent_representations = torch.cat(latent_representations, dim=0)
        glcm_features = torch.tensor(glcm_features)
        lbp_features = torch.tensor(lbp_features)
        
        multi_latents.append(latent_representations)
        multi_glcm.append(glcm_features)
        multi_lbp.append(lbp_features)
        epoch -= 1

    return multi_latents, multi_glcm, multi_lbp, labels, labels_cnt, filenames

In [24]:
# Extract latent representations for the training and validation datasets
train_latents, train_glcm, train_lbp, train_labels, train_labels_cnt, train_filenames = extract_latent_representations(train_loader, mae_model, device, 50)
torch.save({'latents': train_latents, 'glcm': train_glcm, 'lbp': train_lbp, 'labels': train_labels, 'labels_cnt': train_labels_cnt, 'filenames': train_filenames}, 'outputs/train_latents.pth')

valid_latents, valid_glcm, valid_lbp, valid_labels, valid_labels_cnt, valid_filenames = extract_latent_representations(valid_loader, mae_model, device)
torch.save({'latents': valid_latents, 'glcm': valid_glcm, 'lbp': valid_lbp, 'labels': valid_labels, 'labels_cnt': valid_labels_cnt, 'filenames': valid_filenames}, 'outputs/valid_latents.pth')

test_latents, test_glcm, test_lbp, test_labels, test_labels_cnt, test_filenames = extract_latent_representations(test_loader, mae_model, device)
torch.save({'latents': test_latents, 'glcm': test_glcm, 'lbp': test_lbp, 'labels': test_labels, 'labels_cnt': test_labels_cnt, 'filenames': test_filenames}, 'outputs/test_latents.pth')

print(len(train_latents))
print(valid_latents.shape)
print(test_latents.shape)

100%|█████████████████████████████████████████████████████████████████████████████| 674/674 [35:51<00:00,  3.19s/batch]


11252
torch.Size([2814, 768])
torch.Size([21541, 768])
